In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: 4a5e0119-fb85-40fd-87ab-f201a2aa0fde: GSB - UHE-MACHADINHO>, <Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: b75339ff-e91f-4e75-a082-5d7e23a4d049: GSB - UHE-SALTO SANTIAGO>, <Company: ed749424-27e4-4342-82cd-f924de71830b: Limbo - UTPS>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA>, <Company: 7fe7b473-6d9c-49d6-a9ca-74f3424d3211: GSB - UHE-SALTO OSÓRIO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: d25d0eb2-61a8-4646-8e0e-31c8e68cf216: TESTE>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 1b

# IMPORTAR PONTOS DE MONITORAMENTO

In [14]:
from django.db.models import Q
from tqdm.notebook import tqdm
from datetime import datetime
from openpyxl import load_workbook
from django.contrib.gis.geos import Point

In [15]:
def load_excel(filename,sheetPosision=0):
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[wb.sheetnames[sheetPosision]]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value
    #         if 'Foto Contenção 1_1' == key:
    #             print(cell.value)
        data.append(row_dict)
    return data

In [16]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)
    
    lng = pnt.coords[1]
    lat = pnt.coords[0]
    
    return Point(lat, lng)

In [17]:
values=load_excel('piezometros_UHEET')
values[0]

{'Piezômetro': 'PZ 07',
 'Município': 'Carolina',
 'X (SIRGAS 2000)': 226901,
 'Y (SIRGAS 2000)': 9189419,
 'Profund, Instalação (m)': 5.04,
 'Data de Instalação': datetime.datetime(2022, 11, 10, 0, 0),
 'Cota Instalada': 163251,
 'Altura da Base (m)': 0.3}

In [18]:
transform_coordinates(226901,9189419)

In [19]:
mon_plan = MonitoringPlan.objects.get(uuid="f3356e23-0b49-4dd7-81e5-450d6187b9c7")
user=User.objects.get(username="engie")
company=Company.objects.get(name='GES - UHE-ESTREITO')

In [23]:
MonitoringPoint._meta.get_fields()

(<ManyToManyRel: occurrence_records.occurrencerecord>,
 <ManyToManyRel: monitorings.monitoringfrequency>,
 <ManyToOneRel: monitorings.monitoringcollect>,
 <ManyToOneRel: monitorings.monitoringrecord>,
 <django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.DateTimeField: updated_at>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.BooleanField: active>,
 <django.db.models.fields.TextField: code>,
 <django.db.models.fields.IntegerField: uf_code>,
 <django.db.models.fields.related.ForeignKey: city>,
 <django.db.models.fields.related.ForeignKey: location>,
 <django.db.models.fields.related.ForeignKey: river>,
 <django.db.models.fields.CharField: place_on_dam>,
 <django.contrib.gis.db.models.fields.PointField: coordinates>,
 <django.db.models.fields.related.ForeignKey: monitoring_plan>,
 <django.contrib.gis.db.models.fields.PolygonField: coverage_area>,
 <django.db.models.fields.TextFie

In [20]:
mp = MonitoringPoint.objects.filter(monitoring_plan=mon_plan)
mp.count()

112

In [12]:
mp[1].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fef3a0e23d0>,
 'uuid': UUID('ecd2ad1e-3bb9-4f61-bba1-63e5151372e0'),
 'created_at': datetime.datetime(2021, 7, 21, 18, 41, 54, 420690, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2021, 7, 21, 18, 41, 54, 420759, tzinfo=<UTC>),
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'active': True,
 'code': 'PZ 08',
 'uf_code': 21,
 'city_id': UUID('5238f4ca-08fa-41c9-b1d2-ffccc4dc3d43'),
 'location_id': None,
 'river_id': None,
 'place_on_dam': '4',
 'coordinates': <Point object at 0x7fef1d69dd90>,
 'monitoring_plan_id': UUID('f3356e23-0b49-4dd7-81e5-450d6187b9c7'),
 'coverage_area': None,
 'segment': '',
 'description': '',
 'depth': '',
 'position': '',
 'stratification': '',
 'zone': ''}

In [14]:
datetime.today()

datetime.datetime(2023, 9, 13, 13, 39, 29, 335228)

In [22]:
for a in tqdm(values):
    try:
        mpoint =MonitoringPoint.objects.get(monitoring_plan=mon_plan,code=a['Piezômetro'])
        mpoint.updated_at = datetime.today()
        mpoint.city = City.objects.filter(monitoringpoint__monitoring_plan__company=company, name=a['Município']).first()
        mpoint.coordinates = transform_coordinates(a['X (SIRGAS 2000)'],a['Y (SIRGAS 2000)'],srid=31983)
        mpoint.save()
    except Exception as e:
        print(e)
#         mpoint = MonitoringPoint(created_by = user,
#                                 active = True,
#                                 code = a['Piezômetro'],
#                                 city=City.objects.filter(monitoringpoint__monitoring_plan__company=company,
#                                                           name=a['Município']).first(),
#                                 uf_code = City.objects.filter(monitoringpoint__monitoring_plan__company=company,
#                                                           name=a['Município']).first().uf_code,
#                                 place_on_dam = '4',
#                                 coordinates = transform_coordinates(a['X (SIRGAS 2000)'],a['Y (SIRGAS 2000)']),
#                                 monitoring_plan=mon_plan)
#         mpoint.save()

  0%|          | 0/110 [00:00<?, ?it/s]

In [49]:
City.objects.filter(monitoringpoint__monitoring_plan__company=company, name='Carolina').first().uf_code

21